In [ ]:
import nest_asyncio
nest_asyncio.apply()


2.1 Create the Folder Structure

In [ ]:
!mkdir -p agents

2.2 Install Dependencies

In [ ]:
!pip install google-genai==0.2.0 fastapi uvicorn nest_asyncio
!pip install "google-cloud-aiplatform>=1.60.0"


  Using cached google_genai-0.2.0-py3-none-any.whl.metadata (17 kB)
Using cached google_genai-0.2.0-py3-none-any.whl (110 kB)
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.52.0
    Uninstalling google-genai-1.52.0:
      Successfully uninstalled google-genai-1.52.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.18.0 requires google-genai<2.0.0,>=1.45.0, but you have google-genai 0.2.0 which is incompatible.
google-adk 1.18.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 14.2 which is incompatible.
google-cloud-aiplatform 1.125.0 requires google-genai<2.0.0,>=1.37.0, but you have google-genai 0.2.0 which is incompatible.
  Using cached google_genai-1.52.0-py3-none-any.whl.metadata (46 kB)
Using cached google_genai-1.52.0-py3-none-any.whl (261 kB)
  Attempting uninstall: google-genai
    Found ex

2.3 Add Your API Key

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR KEY"


2.4 Create the Requirements File

In [ ]:
%%writefile requirements.txt
google-genai==0.2.0
fastapi
uvicorn
nest_asyncio
google-cloud-aiplatform>=1.60.0


Overwriting requirements.txt


2.5 Verify ADK Installation

In [ ]:
from google import genai
print("ADK is ready.")


ADK is ready.


Now, lets create TaskAgent that will:

Accept a task object from the front-end

Read:

title

description

category

Transform them into normalized structured data

Extract “concepts” or keywords

Estimate complexity

Decide which specialized agent (Quiz / Help / Networking) will handle it

Return JSON that the OrchestratorAgent can use

## 🧠 **Multi-Agent Graph Architecture (3 Core Agents + Router)**

### **1. TaskUnderstandingAgent**

* Input: raw task from UI
* Output: normalized task + metadata
* Predicts: complexity, keywords, required next agent

### **2. QuizAgent**

* We've already built it
* Generates quiz questions
* Formats perfectly (JSON schema)
* Compatible with your React quiz modal

### **3. HelpAgent**

* We've already built it
* Helps user after failing verification

### **4. Router / Orchestrator Agent**

* Looks at normalized metadata:

  * If simple → route to QuizAgent
  * If networking task → route to NameVerificationAgent
  * If task too complex → route directly to HelpAgent
  * If quiz fails → route to HelpAgent



In [ ]:
%%writefile agents/task_agent.py
from google import genai
import json

class TaskAgent:
    """
    Normalizes tasks and determines which agent should run next.
    """

    def __init__(self, api_key: str, model="gemini-2.0-flash"):
        if not api_key:
            raise ValueError("TaskAgent requires a GOOGLE_API_KEY.")
        self.client = genai.Client(api_key=api_key)
        self.model = model

    async def analyze_task(self, task: dict):
        """
        Normalize input task and decide next agent.
        """

        prompt = f"""
You are TaskAgent.

Your job:
- Normalize the task
- Identify category
- Extract keywords
- Estimate complexity (1–10)
- Choose next agent

Return JSON ONLY with the shape:
{{
  "normalizedTitle": "string",
  "normalizedDescription": "string",
  "category": "string",
  "keywords": ["list", "of", "keywords"],
  "complexity": 1-10,
  "nextAgent": "QuizAgent" or "HelpAgent" or "NetworkingAgent"
}}

Task:
Title: {task.get("title")}
Description: {task.get("description")}
Category: {task.get("category")}
"""

        response = await self.client.aio.models.generate_content(
            model=self.model,
            contents=prompt,
            config={"responseMimeType": "application/json"}
        )

        raw = response.text
        print("\n=== RAW TASKAGENT OUTPUT ===\n", raw)

        # Try safe JSON decode
        try:
            return json.loads(raw)
        except Exception:
            print("JSON parsing failed. Falling back to defaults.")

            return {
                "normalizedTitle": task.get("title"),
                "normalizedDescription": task.get("description"),
                "category": task.get("category") or "General",
                "keywords": [],
                "complexity": 5,
                "nextAgent": "QuizAgent"
            }

    async def process_task(self, task: dict):
        """
        Wrapper so OrchestratorAgent can call this uniformly.
        """
        return await self.analyze_task(task)


Overwriting agents/task_agent.py


In [ ]:
import importlib
import agents.task_agent as task_agent_module

# Force reload from disk
importlib.reload(task_agent_module)

TaskAgent = task_agent_module.TaskAgent

In [ ]:
import asyncio

agent = TaskAgent(os.environ["GOOGLE_API_KEY"])

sample_task = {
    "title": "Learn Python Heaps",
    "description": "Understand heaps and solve 3 problems",
    "category": "Learning"
}

async def test_task():
    out = await agent.analyze_task(sample_task)
    print(out)

# Use create_task instead of asyncio.run
task = asyncio.create_task(test_task())
await task



=== RAW TASKAGENT OUTPUT ===
 {
  "normalizedTitle": "Learn Python Heaps",
  "normalizedDescription": "Understand heaps data structure in Python and solve three problems using heaps.",
  "category": "Data Structures and Algorithms",
  "keywords": ["Python", "heaps", "data structures", "algorithms", "priority queues"],
  "complexity": 5,
  "nextAgent": "QuizAgent"
}
{'normalizedTitle': 'Learn Python Heaps', 'normalizedDescription': 'Understand heaps data structure in Python and solve three problems using heaps.', 'category': 'Data Structures and Algorithms', 'keywords': ['Python', 'heaps', 'data structures', 'algorithms', 'priority queues'], 'complexity': 5, 'nextAgent': 'QuizAgent'}


Create:

agents/quiz_agent.py


This agent must:

Receive normalized task data (from TaskAgent / Orchestrator)

Generate quiz questions

Return only JSON

Evaluate answers when the orchestrator passes them

Provide:

score

correct/incorrect

retry suggestions

In [ ]:
%%writefile agents/quiz_agent.py
from google import genai
import json

class QuizAgent:
    """
    Generates quizzes + grades answers.
    """

    def __init__(self, api_key: str, model="gemini-2.0-flash"):
        if not api_key:
            raise ValueError("QuizAgent requires a GOOGLE_API_KEY.")
        self.client = genai.Client(api_key=api_key)
        self.model = model

    async def generate_quiz(self, task: dict):
        """
        Returns a JSON quiz:
        {
          "questions": [
            { "question": "...", "options": [...], "correctIndex": n }
          ]
        }
        """

        prompt = f"""
You are QuizAgent.

Create a 5-question multiple-choice quiz that tests real understanding.

Task:
Title: {task.get("normalizedTitle")}
Description: {task.get("normalizedDescription")}
Keywords: {task.get("keywords")}

Return JSON ONLY in this exact schema:
{{
  "questions": [
    {{
      "question": "string",
      "options": ["A", "B", "C", "D"],
      "correctIndex": 0
    }}
  ]
}}
"""

        response = await self.client.aio.models.generate_content(
            model=self.model,
            contents=prompt,
            config={"responseMimeType": "application/json"}
        )

        raw = response.text
        print("\n=== RAW QUIZ MODEL OUTPUT ===\n", raw, "\n")

        try:
            return json.loads(raw)
        except Exception as e:
            print("QUIZ JSON PARSE FAILED:", e)

            return {
                "questions": [
                    {
                        "question": "Fallback question: Which option is correct?",
                        "options": ["A", "B", "C", "D"],
                        "correctIndex": 0
                    }
                ]
            }

    async def grade_answers(self, quiz: dict, user_answers: list):
        """
        Returns:
        {
          score: float,
          passed: bool,
          details: [...],
          nextAgent: "HelpAgent" or "MemoryAgent"
        }
        """

        questions = quiz.get("questions", [])
        total = len(questions)
        correct = 0
        details = []

        for i, q in enumerate(questions):
            correct_idx = q["correctIndex"]
            user_idx = user_answers[i] if i < len(user_answers) else None
            is_correct = user_idx == correct_idx

            if is_correct:
                correct += 1

            details.append({
                "question": q.get("question"),
                "userAnswer": user_idx,
                "correctAnswer": correct_idx,
                "isCorrect": is_correct
            })

        score = correct / total if total else 0
        passed = score >= 0.8

        return {
            "score": score,
            "passed": passed,
            "details": details,
            "nextAgent": "MemoryAgent" if passed else "HelpAgent"
        }


Overwriting agents/quiz_agent.py


In [ ]:
import importlib
import agents.quiz_agent as quiz_agent_module

# Force reload from disk
importlib.reload(quiz_agent_module)

QuizAgent = quiz_agent_module.QuizAgent

In [ ]:
import asyncio
from agents.quiz_agent import QuizAgent

quiz_agent = QuizAgent(os.environ["GOOGLE_API_KEY"])

sample_task = {
    "normalizedTitle": "Learn Python Heaps",
    "normalizedDescription": "Understand heaps and solve 3 problems",
    "keywords": ["Heap", "Python", "Data Structures"]
}

async def test_quiz():
    print("=== RUNNING QUIZ AGENT TEST ===")

    quiz = await quiz_agent.generate_quiz(sample_task)

    print("\n=== FINAL QUIZ OUTPUT ===")
    print(quiz)

asyncio.run(test_quiz())


=== RUNNING QUIZ AGENT TEST ===

=== RAW QUIZ MODEL OUTPUT ===
 {
  "questions": [
    {
      "question": "Which of the following is NOT a property of a min-heap?",
      "options": [
        "The value of each node is less than or equal to the value of its children.",
        "It is a complete binary tree.",
        "The root node contains the smallest element.",
        "The value of each node is greater than or equal to the value of its children."
      ],
      "correctIndex": 3
    },
    {
      "question": "What is the time complexity of inserting an element into a heap?",
      "options": [
        "O(1)",
        "O(log n)",
        "O(n)",
        "O(n log n)"
      ],
      "correctIndex": 1
    },
    {
      "question": "Which of the following Python modules is commonly used to implement heaps?",
      "options": [
        "collections",
        "heapq",
        "math",
        "random"
      ],
      "correctIndex": 1
    },
    {
      "question": "What is the result of

Build the HelpAgent (Third Agent)

This agent is responsible for:

✔ Giving help when the quiz is failed
✔ Breaking tasks into steps
✔ Giving motivational nudges
✔ Fetching real resources (Google Search Tool) — extra points in Kaggle!
✔ Creating actionable micro-plans
✔ Returning fully structured JSON

This is the “human-friendly” agent judges love.

In [ ]:
%%writefile agents/help_agent.py
import json
import google.genai as genai


def extract_json(raw_text: str):
    """
    Extract JSON from messy LLM output safely.
    """
    if not raw_text:
        raise ValueError("Empty response text")

    cleaned = raw_text.replace("```json", "").replace("```", "").strip()

    start = cleaned.find("{")
    end = cleaned.rfind("}")

    if start == -1 or end == -1:
        raise ValueError("Could not find JSON braces in LLM text.")

    return json.loads(cleaned[start:end+1])


class HelpAgent:
    """
    Generates structured help content:
      - summary
      - actionable steps
      - external resources
    """

    def __init__(self, api_key: str, model="gemini-2.0-flash"):
        if not api_key:
            raise ValueError("HelpAgent requires an API key.")
        self.client = genai.Client(api_key=api_key)
        self.model = model

    async def generate_help(self, task: dict):
        """
        Return structured JSON help content.
        """

        title = task.get("normalizedTitle", task.get("title", ""))
        desc = task.get("normalizedDescription", task.get("description", ""))

        prompt = f"""
You are a productivity assistant.

Return ONLY valid JSON in this exact schema:

{{
  "summary": "short explanation",
  "steps": ["step 1", "step 2", "step 3"],
  "resources": [
    {{"title": "Resource 1", "url": "https://example.com"}},
    {{"title": "Resource 2", "url": "https://example.com"}}
  ]
}}

Task to help with:
Title: {title}
Description: {desc}
"""

        response = await self.client.aio.models.generate_content(
            model=self.model,
            contents=prompt,
            config={
                "responseMimeType": "application/json",
                "responseSchema": {
                    "type": "object",
                    "properties": {
                        "summary": {"type": "string"},
                        "steps": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "resources": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "title": {"type": "string"},
                                    "url": {"type": "string"}
                                },
                                "required": ["title", "url"]
                            }
                        }
                    },
                    "required": ["summary", "steps", "resources"]
                }
            }
        )

        raw = response.text
        print("\n=== RAW HELP MODEL OUTPUT ===\n", raw, "\n")

        # Should already be valid JSON
        try:
            return json.loads(raw)
        except Exception:
            try:
                return extract_json(raw)
            except Exception as e:
                print("JSON parsing failed:", e)
                print("RAW TEXT:\n", raw)
                return {
                    "summary": "Failed to generate help.",
                    "steps": ["Try again later."],
                    "resources": []
                }


Overwriting agents/help_agent.py


In [ ]:
import importlib
import agents.help_agent as help_agent_module

# Force reload from disk
importlib.reload(help_agent_module)

HelpAgent = help_agent_module.HelpAgent


In [ ]:
help_agent = HelpAgent(api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
import asyncio
import json
import os

sample_task = {
    "title": "Learn Python Heaps",
    "description": "Understand heaps and solve 3 practice problems."
}

async def run():
    out = await help_agent.generate_help(sample_task)

    print("\n=== FINAL PARSED OUTPUT ===")
    print(json.dumps(out, indent=2))

asyncio.run(run())


=== RAW HELP MODEL OUTPUT ===
 {
  "summary": "Learn about heaps (also known as priority queues) in Python and practice implementing them.",
  "steps": [
    "Read about Python's `heapq` module and how to use it to implement heaps.",
    "Solve 3 heap-related coding problems of increasing difficulty on platforms like LeetCode or HackerRank.",
    "Review solutions and analyze time and space complexity of your implementations."
  ],
  "resources": [
    {
      "title": "Python heapq Module Documentation",
      "url": "https://docs.python.org/3/library/heapq.html"
    },
    {
      "title": "LeetCode Heap Problems",
      "url": "https://leetcode.com/tag/heap/"
    }
  ]
} 


=== FINAL PARSED OUTPUT ===
{
  "summary": "Learn about heaps (also known as priority queues) in Python and practice implementing them.",
  "steps": [
    "Read about Python's `heapq` module and how to use it to implement heaps.",
    "Solve 3 heap-related coding problems of increasing difficulty on platforms li

In [ ]:
%%writefile agents/memory_agent.py
import json
from datetime import datetime
import google.genai as genai


class MemoryAgent:
    """
    Tracks task history, quiz scores, and mastery trends.
    """

    def __init__(self, api_key: str, storage_path="memory.json", model="gemini-2.0-flash"):
        if not api_key:
            raise ValueError("MemoryAgent requires an API key.")

        self.client = genai.Client(api_key=api_key)
        self.model = model
        self.storage_path = storage_path

        try:
            with open(storage_path, "r") as f:
                self.memory = json.load(f)
        except FileNotFoundError:
            self.memory = {"history": [], "skills": {}}

    def save(self):
        with open(self.storage_path, "w") as f:
            json.dump(self.memory, f, indent=2)

    async def update_from_quiz(self, task: dict, quiz_results: dict):
        """
        Store quiz performance and update mastery level.
        """

        score = quiz_results.get("score", 0)
        passed = quiz_results.get("passed", False)
        title = task.get("normalizedTitle", "Untitled")

        # Log the quiz
        self.memory["history"].append({
            "title": title,
            "score": score,
            "passed": passed,
            "timestamp": datetime.now().isoformat()
        })

        # Update mastery score (between 0 and 1)
        current = self.memory["skills"].get(title, 0.5)
        delta = (score - 0.5) * 0.3
        self.memory["skills"][title] = max(0, min(1, current + delta))

        self.save()

        return {
            "mastery": self.memory["skills"][title],
            "passed": passed,
            "nextAgent": "None" if passed else "HelpAgent"
        }

    async def recommend_next_tasks(self):
        """
        Ask Gemini to suggest future tasks based on mastery levels.
        """

        prompt = f"""
Based on mastery scores:
{json.dumps(self.memory['skills'], indent=2)}

Recommend 3 ideal next tasks.

Return ONLY JSON:
{{
  "recommendations": [
    {{"title": "string", "reason": "string"}},
    {{"title": "string", "reason": "string"}},
    {{"title": "string", "reason": "string"}}
  ]
}}
"""

        response = await self.client.aio.models.generate_content(
            model=self.model,
            contents=prompt,
            config={
                "responseMimeType": "application/json"
            }
        )

        return json.loads(response.text)


Overwriting agents/memory_agent.py


In [ ]:
import importlib
import agents.memory_agent as memory_agent_module

# Force reload from disk
importlib.reload(memory_agent_module)

MemoryAgent = memory_agent_module.MemoryAgent

In [ ]:
import asyncio
memory_agent = MemoryAgent(api_key = os.getenv("GOOGLE_API_KEY"), storage_path="test_memory.json")

sample_task = {"normalizedTitle": "Learn Python Heaps"}

sample_quiz_results = {
    "score": 0.8,
    "passed": True
}

async def run_memory_agent():
    print("=== RUNNING MEMORY AGENT TEST ===")

    update = await memory_agent.update_from_quiz(sample_task, sample_quiz_results)

    print("\n=== MEMORY UPDATE ===")
    print(update)

    print("\n=== FILE CONTENTS ===")
    import json
    print(json.load(open("test_memory.json")))

asyncio.run(run_memory_agent())


=== RUNNING MEMORY AGENT TEST ===

=== MEMORY UPDATE ===
{'mastery': 1, 'passed': True, 'nextAgent': 'None'}

=== FILE CONTENTS ===
{'history': [{'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T08:55:34.134302'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:08:23.674831'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:11:03.421004'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:16:14.207939'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:17:36.958379'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:20:56.918877'}, {'title': 'Learn Python Heaps', 'score': 0.8, 'passed': True, 'timestamp': '2025-12-01T09:56:49.387235'}], 'skills': {'Learn Python Heaps': 1}}


In [ ]:
%%writefile agents/orchestrator_agent.py
import os
from agents.task_agent import TaskAgent
from agents.quiz_agent import QuizAgent
from agents.help_agent import HelpAgent
from agents.memory_agent import MemoryAgent


class OrchestratorAgent:
    """
    Controls the entire multi-agent workflow.
    """

    def __init__(self):
        api_key = os.getenv("GOOGLE_API_KEY")
        if not api_key:
            raise ValueError("GOOGLE_API_KEY not set in environment.")

        # Pass API key to all agents
        self.task_agent = TaskAgent(api_key=api_key)
        self.quiz_agent = QuizAgent(api_key=api_key)
        self.help_agent = HelpAgent(api_key=api_key)
        self.memory_agent = MemoryAgent(api_key=api_key)

    async def process(self, raw_task: dict):
        """
        STEP 1 — Normalize Task
        STEP 2 — Route to correct agent
        """

        print("\n=== ORCHESTRATOR STEP: TaskAgent ===")

        normalized = await self.task_agent.process_task(raw_task)
        print("Normalized:", normalized)

        next_agent = normalized.get("nextAgent", "HelpAgent")

        # -------------------------
        # ROUTE: QUIZ
        # -------------------------
        if next_agent == "QuizAgent":
            print("\n=== ORCHESTRATOR STEP: QuizAgent (Generating Quiz) ===")
            quiz = await self.quiz_agent.generate_quiz(normalized)

            return {
                "stage": "quiz",
                "normalized": normalized,
                "quiz": quiz
            }

        # -------------------------
        # ROUTE: HELP
        # -------------------------
        print("\n=== ORCHESTRATOR STEP: HelpAgent ===")
        help_data = await self.help_agent.generate_help(normalized)

        return {
            "stage": "help",
            "normalized": normalized,
            "help": help_data
        }

    async def submit_quiz(self, normalized_task: dict, quiz: dict, user_answers: list):
        """
        After quiz: grade → memory or help
        """

        print("\n=== ORCHESTRATOR STEP: Grade Quiz ===")
        result = await self.quiz_agent.grade_answers(quiz, user_answers)
        print("Quiz results:", result)

        # PASS
        if result["passed"]:
            print("\n=== ORCHESTRATOR STEP: MemoryAgent ===")
            mem = await self.memory_agent.update_from_quiz(normalized_task, result)

            return {
                "stage": "memory",
                "memory": mem
            }

        # FAIL → HELP
        print("\n=== ORCHESTRATOR STEP: HelpAgent ===")
        help_data = await self.help_agent.generate_help(normalized_task)

        return {
            "stage": "help",
            "help": help_data
        }


Overwriting agents/orchestrator_agent.py


In [ ]:
import importlib
import agents.orchestrator_agent as orch_agent_module

# Force reload from disk
importlib.reload(orch_agent_module)

OrchAgent = orch_agent_module.OrchestratorAgent

In [ ]:
import asyncio
from agents.orchestrator_agent import OrchestratorAgent

orch = OrchestratorAgent()

sample_task = {
    "title": "Learn Python Heaps",
    "description": "Understand heap operations and solve 3 problems",
    "category": "Learning"
}

async def test():
    print("⇒ Processing new task...")

    # STEP 1: TaskAgent → QuizAgent
    step1 = await orch.process(sample_task)
    print("\n=== STEP 1 OUTPUT ===")
    print(step1)

    # simulated user answers
    user_answers = [0, 1, 2, 3, 0]

    # STEP 2: QuizAgent → memory/help
    print("\n=== QUIZ RESULTS ===")
    quiz_results = await orch.quiz_agent.grade_answers(
        step1["quiz"],
        user_answers
    )
    print(quiz_results)

    # orchestrator handles route after quiz
    step2 = await orch.submit_quiz(
        step1["normalized"],
        step1["quiz"],
        user_answers
    )

    print("\n=== STEP 2 OUTPUT ===")
    print(step2)

asyncio.run(test())


⇒ Processing new task...

=== ORCHESTRATOR STEP: TaskAgent ===

=== RAW TASKAGENT OUTPUT ===
 {
  "normalizedTitle": "Learn Python Heaps",
  "normalizedDescription": "Understand heap operations and solve 3 problems",
  "category": "Learning",
  "keywords": ["Python", "Heaps", "Data Structures", "Algorithms"],
  "complexity": 5,
  "nextAgent": "QuizAgent"
}
Normalized: {'normalizedTitle': 'Learn Python Heaps', 'normalizedDescription': 'Understand heap operations and solve 3 problems', 'category': 'Learning', 'keywords': ['Python', 'Heaps', 'Data Structures', 'Algorithms'], 'complexity': 5, 'nextAgent': 'QuizAgent'}

=== ORCHESTRATOR STEP: QuizAgent (Generating Quiz) ===

=== RAW QUIZ MODEL OUTPUT ===
 {
  "questions": [
    {
      "question": "Which of the following is NOT a property of a min-heap?",
      "options": [
        "The value of each node is greater than or equal to the value of its parent.",
        "It is a complete binary tree.",
        "The root node contains the small

In [ ]:
%%writefile server.py
from fastapi import FastAPI
from agents.orchestrator_agent import OrchestratorAgent

app = FastAPI()
orchestrator = OrchestratorAgent()

@app.post("/api/task")
async def process_task(task: dict):
    return await orchestrator.process(task)

@app.post("/api/quiz/submit")
async def submit_quiz(payload: dict):
    return await orchestrator.handle_quiz_results(
        payload["task"],
        payload["quizResults"]
    )


Overwriting server.py


In [ ]:
!zip -r agents2.zip agents/

  adding: agents/ (stored 0%)
  adding: agents/__pycache__/ (stored 0%)
  adding: agents/__pycache__/quiz_agent.cpython-311.pyc (deflated 43%)
  adding: agents/__pycache__/help_agent.cpython-311.pyc (deflated 43%)
  adding: agents/__pycache__/memory_agent.cpython-311.pyc (deflated 47%)
  adding: agents/__pycache__/task_agent.cpython-311.pyc (deflated 45%)
  adding: agents/__pycache__/orchestrator_agent.cpython-311.pyc (deflated 53%)
  adding: agents/memory_agent.py (deflated 60%)
  adding: agents/task_agent.py (deflated 57%)
  adding: agents/help_agent.py (deflated 66%)
  adding: agents/orchestrator_agent.py (deflated 71%)
  adding: agents/quiz_agent.py (deflated 62%)
